In [1]:
###use this notebook to create databases

In [2]:
from os import system as bash
from pyspark.sql import SparkSession
from splicemachine.spark import PySpliceContext
import pandas as pd
import time

In [3]:
spark = SparkSession.builder.getOrCreate()
sc = spark.sparkContext
splice = PySpliceContext(spark)

In [14]:
days = {'jan':31, 'feb':28, 'march':31, 'april':30, 'may':31, 'june': 30, 'july':31, 'aug':31, 'sep': 30, 'oct': 31, 'nov': 30, 'dec': 31}

In [15]:
def download_file(file):
    bash(f'wget https://coast.noaa.gov/htdata/CMSP/AISDataHandler/2019/{file}.zip')
    bash(f'unzip {file}.zip')
    bash(f'hdfs dfs -copyFromLocal {file}.csv /tmp/file.csv') # Copy to HDFS
    bash(f'rm {file}.zip') # Remove local copy
    bash(f'rm {file}.csv') # Remove local copy
    

In [16]:
########CAREFULLLL---- DO NOT RUN  CELL BELOW. IT WILL OVERWRITE##########
#create table janfebmarch (mmsi int, basedatetime timestamp, lat float, lon float, sog float, cog float, heading float, vesselname varchar(100), imo varchar(100), callsign varchar(100), vesseltype float, status float, length float, width float, draft float, cargo float, transcieverclass varchar(100))


In [8]:
%%sql 
drop table if exists february;
create table february (mmsi int, basedatetime timestamp, lat float, lon float)

Sql started successfully



In [10]:
schema = splice.getSchema('january')

In [10]:
schema

StructType(List(StructField(MMSI,IntegerType,true),StructField(BASEDATETIME,TimestampType,true),StructField(LAT,DoubleType,true),StructField(LON,DoubleType,true)))

In [4]:
df = splice.df("select * from AGUPTA.janfebmarch")

In [7]:
help(df)

Help on DataFrame in module pyspark.sql.dataframe object:

class DataFrame(builtins.object)
 |  DataFrame(jdf, sql_ctx)
 |  
 |  A distributed collection of data grouped into named columns.
 |  
 |  A :class:`DataFrame` is equivalent to a relational table in Spark SQL,
 |  and can be created using various functions in :class:`SparkSession`::
 |  
 |      people = spark.read.parquet("...")
 |  
 |  Once created, it can be manipulated using the various domain-specific-language
 |  (DSL) functions defined in: :class:`DataFrame`, :class:`Column`.
 |  
 |  To select a column from the data frame, use the apply method::
 |  
 |      ageCol = people.age
 |  
 |  A more concrete example::
 |  
 |      # To create DataFrame using SparkSession
 |      people = spark.read.parquet("...")
 |      department = spark.read.parquet("...")
 |  
 |      people.filter(people.age > 30).join(department, people.deptId == department.id) \
 |        .groupBy(department.name, "gender").agg({"salary": "avg", "age

In [6]:
dp = df.head(5).toPandas()

AttributeError: 'list' object has no attribute 'toPandas'

In [ ]:
dp

In [9]:
SCHEMA = 'AGUPTA'
TABLE = 'january'

In [18]:
for i, month in enumerate(['feb',]):
    for day in range(days[month]):
        file = 'AIS_2019_' + f'{i+1:02}'  + '_' f'{day + 1:02}'
        print('Downloading file:', file)
        download_file(file)
        
        df = spark.read.option("timestampFormat", "yyyy-MM-dd HH:mm:ss").csv(f'/tmp/file.csv', header=True, schema=schema)
        df = df['MMSI', 'BASEDATETIME', 'LAT', 'LON']
        splice.insert(df, f'{SCHEMA}.{TABLE}', to_upper=True)
        print('Done')

Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done


In [11]:
spark.read.option("timestampFormat", "yyyy-MM-dd HH:mm:ss").csv(f'/tmp/file.csv', header=True, schema=schema).show(10)

+---------+-------------------+--------+----------+----+------+-------+----------+----+--------+----------+------+------+-----+-----+-----+----------------+
|     MMSI|       BASEDATETIME|     LAT|       LON| SOG|   COG|HEADING|VESSELNAME| IMO|CALLSIGN|VESSELTYPE|STATUS|LENGTH|WIDTH|DRAFT|CARGO|TRANSCIEVERCLASS|
+---------+-------------------+--------+----------+----+------+-------+----------+----+--------+----------+------+------+-----+-----+-----+----------------+
|338208626|2019-01-01T00:00:05|37.86505|-122.31405| 0.0|-165.0|  511.0|      null|null|    null|      null|     0|  null| null| null| null|               B|
|367420490|2019-01-01T00:00:04|31.26095| -91.65160| 4.5|  15.0|   31.0|      null|null|    null|      null|    15|  null| null| null| null|               B|
|338060122|2019-01-01T00:00:04|30.67109| -81.46722| 1.9| -67.4|  511.0|      null|null|    null|      null|     0|  null| null| null| null|               B|
|368018190|2019-01-01T00:00:04|30.70736| -88.04332| 0.0|-1

In [41]:
#Alternate way to define schema
'''from pyspark.sql.types import *
schema = StructType([StructField('mmsi', StringType(), True),
                     StructField('basedatetime', StringType(), True),
                     StructField('lat', StringType(), True),
                     StructField('lon', StringType(), True),
                    StructField('sog', StringType(), True),
                    StructField('cog', StringType(), True),
                    StructField('heading', StringType(), True),
                    StructField('vesselname', StringType(), True),
                    StructField('imo', StringType(), True),
                    StructField('callsign', StringType(), True),
                    StructField('vesseltype', StringType(), True),
                    StructField('status', StringType(), True),
                    StructField('length', StringType(), True),
                    StructField('width', StringType(), True),
                    StructField('draft', StringType(), True),
                    StructField('cargo', StringType(), True),
                    StructField('transcieverclass', StringType(), True)])'''

"from pyspark.sql.types import *\nschema = StructType([StructField('mmsi', StringType(), True),\n                     StructField('basedatetime', StringType(), True),\n                     StructField('lat', StringType(), True),\n                     StructField('lon', StringType(), True),\n                    StructField('sog', StringType(), True),\n                    StructField('cog', StringType(), True),\n                    StructField('heading', StringType(), True),\n                    StructField('vesselname', StringType(), True),\n                    StructField('imo', StringType(), True),\n                    StructField('callsign', StringType(), True),\n                    StructField('vesseltype', StringType(), True),\n                    StructField('status', StringType(), True),\n                    StructField('length', StringType(), True),\n                    StructField('width', StringType(), True),\n                    StructField('draft', StringType(), True),\n    

In [4]:
df = splice.df("select * from AGUPTA.january")

In [7]:
df.head(10)

[Row(MMSI='367387930', BASEDATETIME='2019-01-01T10:15:59', LAT='29.12124', LON='-94.56475', SOG='1.2', COG='-201.4', HEADING='511.0', VESSELNAME='ODYSSEA DEFENDER', IMO='IMO9535072', CALLSIGN='WDE7022', VESSELTYPE='90', STATUS='0', LENGTH='58', WIDTH='12', DRAFT='4.7', CARGO='79', TRANSCIEVERCLASS='B'),
 Row(MMSI='311000500', BASEDATETIME='2019-01-01T10:15:45', LAT='28.79222', LON='-89.36088', SOG='2.8', COG='78.6', HEADING='70.0', VESSELNAME='AMAZON BRILLIANCE', IMO='IMO9294678', CALLSIGN='C6CK8', VESSELTYPE='80', STATUS='0', LENGTH='228', WIDTH='40', DRAFT='12.0', CARGO='80', TRANSCIEVERCLASS='B'),
 Row(MMSI='366984870', BASEDATETIME='2019-01-01T19:32:28', LAT='38.38538', LON='-90.34396', SOG='4.1', COG='32.0', HEADING='33.0', VESSELNAME='DAN MACMILLAN', IMO=None, CALLSIGN='WDB9903', VESSELTYPE='31', STATUS='12', LENGTH='53', WIDTH='16', DRAFT=None, CARGO='31', TRANSCIEVERCLASS='B'),
 Row(MMSI='367940000', BASEDATETIME='2019-01-01T00:00:05', LAT='41.46016', LON='-70.73666', SOG='12.9